## Training Materials Outline: ALOS-2 Data Preprocessing and Time-Series Analysis

**Module 1: Introduction to ALOS-2 Data and Applications**

* 1.1 What is ALOS-2?
    * Overview of the ALOS-2 satellite and its sensors (PALSAR-2).
    * Key features and advantages of ALOS-2 data.
* 1.2 Applications of ALOS-2 Data
    * Focus on forest monitoring, deforestation detection, and oil palm plantation mapping.
    * Examples of real-world applications.
* 1.3 Data Access and Formats
    * Sources for downloading ALOS-2 data (JAXA, ASF DAAC).
    * Explanation of common data formats (GeoTIFF, HDF5).

**Module 2: Basic Python for Earth Observation Data Processing**

* 2.1 Introduction to Python Libraries
    * `rasterio` for raster data handling.
    * `geopandas` for vector data and shapefiles.
    * `numpy` for numerical computations.
    * `matplotlib` for visualization.
* 2.2 Python Fundamentals
    * Reading and writing files.
    * Defining functions.
    * Basic data manipulation with `numpy` arrays.

**Module 3: ALOS-2 Data Preprocessing**

* 3.1 Loading ALOS-2 Data with `rasterio`
    * Opening GeoTIFF files.
    * Reading image data into `numpy` arrays.
    * Accessing metadata (e.g., transform, CRS).
* 3.2 Clipping Data to an Area of Interest (AOI)
    * Loading shapefiles with `geopandas`.
    * Using `rasterio.mask` to clip raster data.
    * Updating metadata after clipping.
* 3.3 Radiometric Calibration
    * Applying the calibration formula: `(20 * np.log10(DN)) - 83`.
    * Explanation of the calibration process and its importance.
* 3.4 Speckle Filtering with the Lee Filter
    * Implementing the Lee filter function in Python.
    * Applying the filter to the calibrated image.
    * Visualizing the effects of filtering.
* 3.5 Saving Preprocessed Data
    * Writing processed data to new GeoTIFF files using `rasterio`.
    * Updating metadata to reflect changes (e.g., data type).

In [ ]:
import rasterio
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from rasterio.mask import mask
from scipy.ndimage import uniform_filter

def lee_filter(image, size=5):
    mean = uniform_filter(image, (size, size))
    mean_sq = uniform_filter(image**2, (size, size))
    variance = mean_sq - mean**2
    overall_variance = np.var(image)
    noise_variance = np.maximum(variance - overall_variance, 0)
    weights = noise_variance / (noise_variance + variance)
    filtered_image = mean + weights * (image - mean)
    return filtered_image

# Example usage (replace with your actual file paths)
image_path = 'example.tif' # Replace with your image path
aoi_path = 'aoi.shp' # Replace with your AOI path

with rasterio.open(image_path) as src:
    aoi = gpd.read_file(aoi_path)
    out_image, out_transform = mask(src, aoi.geometry, crop=True)
    out_meta = src.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform
    })
    clipped_image = out_image[0]
    calibrated_image = (20 * np.log10(clipped_image)) - 83
    filtered_image = lee_filter(calibrated_image)

    # Example visualization
    plt.imshow(filtered_image, cmap='gray')
    plt.show()

    # Example saving
    with rasterio.open('filtered_image.tif', 'w', **out_meta) as dest:
        dest.write(filtered_image, indexes=1)

**Module 4: Time-Series Analysis of ALOS-2 Data**

* 4.1 Preparing Time-Series Data
    * Preprocessing multiple ALOS-2 images from different dates.
    * Layer stacking using `numpy.stack` or `rasterio.merge`.
    * Creating a multi-band raster representing the time series.
* 4.2 Visualizing Time-Series Data in QGIS
    * Loading the stacked raster into QGIS.
    * Creating RGB composites to visualize change.
    * Using the Temporal/Spectral Profile Tool to analyze change at specific locations.
* 4.3 Interpreting Change Detection Results
    * Identifying patterns of deforestation and oil palm plantation expansion.
    * Comparing ALOS-2 data with optical imagery (e.g., Landsat, Sentinel-2).
    * Understanding the limitations of SAR data for change detection.

**Module 5: Advanced Topics (Optional)**

* 5.1 Other Speckle Filters (e.g., Frost filter, Refined Lee filter).
* 5.2 Geometric Correction.
* 5.3 Introduction to SAR Polarimetry and its applications.